In [1]:
import streamlit as st
import time
import joblib
import simpy

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import openpyxl
import tempfile

# from models.VOVO import *
import random

In [2]:
from utils.helpers import *

In [3]:
attrition_model = joblib.load("./models/attrition_model.joblib")

In [4]:
target_capacity = pd.DataFrame(index = pd.date_range('07/03/2023', '12/26/2023', freq = 'W'))
target_capacity['values'] = 3000

In [5]:
# temporaty! define ramping function
def ramping_function(week, ppr_list): 
    for w, tickets in enumerate(ppr_list): 
        if week == w: 
            return tickets
    return ppr_list[-1]

ppr_list = [13, 26, 39, 65]
ramping_function_sched = lambda n : ramping_function(n, ppr_list = ppr_list)

In [8]:
def Simulation(target_capacity, 
               ramping_function = ramping_function_sched,
               attrition_model = attrition_model):
    
    start_date = target_capacity.index.min()
    end_date = target_capacity.index.max()
    target_capacity = target_capacity.values
    N = np.ceil(target_capacity[0] / ramping_function(5))
    mode = 'capacity'
    
    n_weeks = 12

    sim_len = (end_date - start_date).days // 7    
    record = np.zeros((sim_len, n_weeks + sim_len + 2))

    env = simpy.Environment()
    team = Team(env, attrition_model, ramping_function)
    
    env.process(
        sustain(env = env, N = N, record = record, start_date = start_date, end_date = end_date, n_weeks = n_weeks, target_capacity = target_capacity,
                team = team, mode = mode)
    )
    env.run(sim_len)
    
    rf = np.vectorize(ramping_function)
    period = pd.date_range(start_date, end_date, freq = 'W', inclusive = 'left')
    df = pd.DataFrame(record)
    df = df.assign(
        global_capacity = np.dot(df.iloc[:, :-2].values, rf(df.iloc[:, :-2].columns).T),
        global_headcount = (df.iloc[:, -2] + df.iloc[:, -1]), 
        vovo_headcount = df.iloc[:, -2], 
        remote_headcount = df.iloc[:, -1], 
        date = period
    )
    
    r, c = record.shape
    df['weekly_attrition'] = df[0]

    LL_agg = df.copy()

    LL_agg['month'] = LL_agg.date.dt.month
    LL_agg['Year'] = LL_agg.date.dt.year
    LL_agg['Quarter'] = np.nan
    LL_agg.loc[LL_agg.month == 1, 'Quarter'] = 1
    LL_agg.loc[LL_agg.month == 4, 'Quarter'] = 2
    LL_agg.loc[LL_agg.month == 7, 'Quarter'] = 3
    LL_agg.loc[LL_agg.month == 10, 'Quarter'] = 4

    LL_agg = LL_agg.fillna(method = 'ffill')
    LL_agg['last_q'] = LL_agg['Quarter'].shift(1)

    LL_agg['cum_tix'] = LL_agg.groupby(['Quarter', 'Year']).global_capacity.cumsum()
    LL_agg_loc = LL_agg.copy()

    LL_agg_loc['total_team_size'] = LL_agg_loc.loc[:, 0:c-1].sum(axis = 1)
    LL_agg_loc['5+'] = LL_agg_loc.loc[:, 4:c-1].sum(axis = 1)
    LL_agg_loc['date'] = pd.to_datetime(LL_agg_loc['date']).dt.date

    return LL_agg, LL_agg_loc
    # return df, initial_headcount

In [9]:
LL_agg, LL_agg_loc = Simulation(target_capacity)

[3055.]


In [2]:
import gspread
import pandas as pd

In [12]:
sheetid = '1NeB007_XiylxDMfw1pCJ69b0VtO2L8VwBC_V6oZTZew'
sheetname = 'Sheet1'

gc = gspread.service_account('rsr-sandbox-sbx-3a2f-a12b8bf825d2.json')

spreadsheet = gc.open_by_key(sheetid)

worksheet = spreadsheet.worksheet(sheetname)
rows = worksheet.get_all_records()

In [13]:
df = pd.DataFrame(rows)

In [14]:
df

,A,B,C
0,9,7,7
1,5,4,2
